# import and read

In [2]:
import pandas as pd
import os
import sqlalchemy as sql
import numpy as np
import sqlite3

In [3]:
# Pfad zur Excel-Datei
file_path = 'C:\\Users\\hp\\OneDrive\\Desktop\\DBU\\SP3-1.Dateninfrastrukturen\\Pruefung\\Zensus\\input\\Zensus.xlsx'
# Lese die Excel-Datei ein und speichere die Blätter als einzelne DataFrames
sheets = pd.read_excel(file_path, sheet_name=None)
data_frames = {sheet_name: data for sheet_name, data in sheets.items() if sheet_name.startswith('CSV')}

# check
print("Dateninfrastruktur aufgesetzt und Excel-Daten geladen.")

Dateninfrastruktur aufgesetzt und Excel-Daten geladen.


In [4]:
# Den Pfad kontrollieren
db_path = os.path.abspath('zensus.db')
print(f"Die SQLite-Datenbank wurde hier erstellt: {db_path}")

Die SQLite-Datenbank wurde hier erstellt: c:\Users\hp\OneDrive\Desktop\DBU\SP3-1.Dateninfrastrukturen\Pruefung\Zensus\notebooks\zensus.db


In [5]:
# Speichern der DataFrames in separaten Variablen mit angepassten Namen
created_dfs = {}
for sheet_name, data in data_frames.items():
    new_sheet_name = sheet_name.replace('-', '_')
    df = data.replace(['.', '/','_'], np.nan)  # Ersetzen von '.','_ und '/' durch NaN
    for col in df.columns:
        if 'STP' in col:
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Umwandlung in numerische Typen
    globals()[new_sheet_name] = df
    created_dfs[new_sheet_name] = df

# Anzeigen der neuen Namen der erstellten DataFrames
print("Erstellte DataFrames:")
for df_name in created_dfs.keys():
    print(df_name)

C:\Users\hp\AppData\Local\Temp\ipykernel_25500\3764804539.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = data.replace(['.', '/','_'], np.nan)  # Ersetzen von '.','_ und '/' durch NaN


Erstellte DataFrames:
CSV_Klassenstufe
CSV_Schulform
CSV_Hoechster_Schulabschluss
CSV_Hoechster_berufl_Abschluss
CSV_Erwerbsstatus
CSV_ET_Alter
CSV_ET_Hoechst_berufl_Abschl
CSV_ET_Stellung_im_Beruf
CSV_ET_Beruf_Hauptgr_ISCO08
CSV_ET_Wirtschaftszweig


In [6]:
# Überprüfung auf fehlende Werte in allen DataFrames und Konvertierung in numerische Typen
for i, df_name in enumerate(created_dfs):
    df = globals()[df_name]
    print(f"Missing values in DataFrame {i + 1} ({df_name}):")
    print(df.isnull().sum())
    for col in df.columns:
        if 'STP' in col:
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Umwandlung in numerische Typen
    globals()[df_name] = df  # Aktualisieren des DataFrames in der globalen Variablen

Missing values in DataFrame 1 (CSV_Klassenstufe):
Berichtszeitpunkt             0
_RS                           0
Name                          0
Regionalebene                 0
SCHUELER_KLSS_STP          9812
SCHUELER_KLSS_STP__M       9812
SCHUELER_KLSS_STP__W       9812
SCHUELER_KLSS_STP__1       9813
SCHUELER_KLSS_STP__1_M    10144
SCHUELER_KLSS_STP__1_W    10240
SCHUELER_KLSS_STP__2       9812
SCHUELER_KLSS_STP__2_M     9841
SCHUELER_KLSS_STP__2_W     9851
SCHUELER_KLSS_STP__3      10493
SCHUELER_KLSS_STP__3_M    11370
SCHUELER_KLSS_STP__3_W    11275
dtype: int64
Missing values in DataFrame 2 (CSV_Schulform):
Berichtszeitpunkt            0
_RS                          0
Name                         0
Regionalebene                0
SCHUELER_SCH_STP          9812
SCHUELER_SCH_STP__M       9812
SCHUELER_SCH_STP__W       9812
SCHUELER_SCH_STP__1       9813
SCHUELER_SCH_STP__1_M    10185
SCHUELER_SCH_STP__1_W    10264
SCHUELER_SCH_STP__2      11906
SCHUELER_SCH_STP__2_M    12046
SCHUEL

# EDA

In [7]:
for df_name, df in created_dfs.items():
    print(f"Spalten in {df_name}:")
    print(df.columns)
    print("\n")

Spalten in CSV_Klassenstufe:
Index(['Berichtszeitpunkt', '_RS', 'Name', 'Regionalebene',
       'SCHUELER_KLSS_STP', 'SCHUELER_KLSS_STP__M', 'SCHUELER_KLSS_STP__W',
       'SCHUELER_KLSS_STP__1', 'SCHUELER_KLSS_STP__1_M',
       'SCHUELER_KLSS_STP__1_W', 'SCHUELER_KLSS_STP__2',
       'SCHUELER_KLSS_STP__2_M', 'SCHUELER_KLSS_STP__2_W',
       'SCHUELER_KLSS_STP__3', 'SCHUELER_KLSS_STP__3_M',
       'SCHUELER_KLSS_STP__3_W'],
      dtype='object')


Spalten in CSV_Schulform:
Index(['Berichtszeitpunkt', '_RS', 'Name', 'Regionalebene', 'SCHUELER_SCH_STP',
       'SCHUELER_SCH_STP__M', 'SCHUELER_SCH_STP__W', 'SCHUELER_SCH_STP__1',
       'SCHUELER_SCH_STP__1_M', 'SCHUELER_SCH_STP__1_W', 'SCHUELER_SCH_STP__2',
       'SCHUELER_SCH_STP__2_M', 'SCHUELER_SCH_STP__2_W', 'SCHUELER_SCH_STP__3',
       'SCHUELER_SCH_STP__3_M', 'SCHUELER_SCH_STP__3_W', 'SCHUELER_SCH_STP__4',
       'SCHUELER_SCH_STP__4_M', 'SCHUELER_SCH_STP__4_W', 'SCHUELER_SCH_STP__5',
       'SCHUELER_SCH_STP__5_M', 'SCHUELER_SCH

In [8]:
CSV_Klassenstufe.groupby('Regionalebene')['SCHUELER_KLSS_STP'].sum()

Regionalebene
Bund                                     8903780.0
Gemeinde                                 6631640.0
Gemeindeverband                          1310750.0
Land                                     8903780.0
Regierungsbezirk                         7013880.0
Stadtkreis/kreisfreie Stadt/Landkreis    8903770.0
Name: SCHUELER_KLSS_STP, dtype: float64

# transform

## 

## regionalebenen

In [9]:
# Definieren der Regionalebenen
liste = [
    'Bund',
    'Gemeinde',
    'Gemeindeverband',
    'Land',
    'Regierungsbezirk',
    'Stadtkreis/kreisfreie Stadt/Landkreis'
]

In [10]:
# Erstellen der Tabellen für alle Regionalebenen
regionalebenen = {}
for ebene in liste:
    regionalebenen[ebene] = pd.DataFrame()
    for df in created_dfs.values():
        if 'Regionalebene' in df.columns:
            filtered_df = df[df['Regionalebene'] == ebene][['Berichtszeitpunkt', '_RS', 'Name']]
            regionalebenen[ebene] = pd.concat([regionalebenen[ebene], filtered_df], ignore_index=True)
    regionalebenen[ebene] = regionalebenen[ebene].drop_duplicates().reset_index(drop=True)
    print(f"Tabelle für {ebene} wurde erfolgreich erstellt.")
    print(regionalebenen[ebene].head())

Tabelle für Bund wurde erfolgreich erstellt.
   Berichtszeitpunkt  _RS         Name
0           20220515    0  Deutschland
Tabelle für Gemeinde wurde erfolgreich erstellt.
   Berichtszeitpunkt          _RS                    Name
0           20220515  10010000000        Flensburg, Stadt
1           20220515  10020000000  Kiel, Landeshauptstadt
2           20220515  10030000000      Lübeck, Hansestadt
3           20220515  10040000000       Neumünster, Stadt
4           20220515  10510011011      Brunsbüttel, Stadt
Tabelle für Gemeindeverband wurde erfolgreich erstellt.
   Berichtszeitpunkt       _RS                    Name
0           20220515  10515163  Burg-St. Michaelisdonn
1           20220515  10515166           Marne-Nordsee
2           20220515  10515169                   Eider
3           20220515  10515172           Heider Umland
4           20220515  10515175      Mitteldithmarschen
Tabelle für Land wurde erfolgreich erstellt.
   Berichtszeitpunkt  _RS                 Name
0 

In [11]:
# Beispiel für den Zugriff auf eine spezifische Ebene
gemeinde_df = regionalebenen['Gemeinde']
print("Beispiel für die Gemeindetabelle:")
print(gemeinde_df.head())

Beispiel für die Gemeindetabelle:
   Berichtszeitpunkt          _RS                    Name
0           20220515  10010000000        Flensburg, Stadt
1           20220515  10020000000  Kiel, Landeshauptstadt
2           20220515  10030000000      Lübeck, Hansestadt
3           20220515  10040000000       Neumünster, Stadt
4           20220515  10510011011      Brunsbüttel, Stadt


In [12]:
for ebene in regionalebenen: regionalebenen[ebene] = regionalebenen[ebene].drop(columns=['Berichtszeitpunkt'])

In [13]:
# Erstellen der Spalten für Gemeindeverbandsschlüssel (erste 8 Stellen)
for ebene in ['Gemeinde', 'Gemeindeverband']:
    regionalebenen[ebene]['Gemeindeverbandsschlüssel'] = regionalebenen[ebene]['_RS'].astype(str).str[:8].astype(int)

# Erstellen der Spalten für Kreisschlüssel (erste 4 Stellen)
for ebene in ['Gemeindeverband', 'Stadtkreis/kreisfreie Stadt/Landkreis']:
    regionalebenen[ebene]['Kreisschlüssel'] = regionalebenen[ebene]['_RS'].astype(str).str[:4].astype(int)

# Erstellen der Spalten für Landesschlüssel (erste 2 Stellen)
for ebene in ['Stadtkreis/kreisfreie Stadt/Landkreis', 'Land']:
    regionalebenen[ebene]['Landesschlüssel'] = regionalebenen[ebene]['_RS'].astype(str).str[:2].astype(int)

In [14]:
# Alle Tabellen ohne die Spalte '_RS' speichern (ausser Gemeinde)
for ebene, df in regionalebenen.items():
    if ebene != 'Gemeinde':
        regionalebenen[ebene] = df.drop('_RS', axis=1)

# _RS in Gemeinde umbenennen
regionalebenen['Gemeinde'] = regionalebenen['Gemeinde'].rename(columns={'_RS': 'Gemeindeschlüssel'})

In [15]:
# Überprüfen der ersten Zeilen der Tabellen

print("Gemeindetabelle:")
print(regionalebenen['Gemeinde'].head())

print("\nGemeindeverbandstabelle:")
print(regionalebenen['Gemeindeverband'].head())

print("\nKreis:")
print(regionalebenen['Stadtkreis/kreisfreie Stadt/Landkreis'].head())

print("\nLändertabelle:")
print(regionalebenen['Land'].head())

Gemeindetabelle:
   Gemeindeschlüssel                    Name  Gemeindeverbandsschlüssel
0        10010000000        Flensburg, Stadt                   10010000
1        10020000000  Kiel, Landeshauptstadt                   10020000
2        10030000000      Lübeck, Hansestadt                   10030000
3        10040000000       Neumünster, Stadt                   10040000
4        10510011011      Brunsbüttel, Stadt                   10510011

Gemeindeverbandstabelle:
                     Name  Gemeindeverbandsschlüssel  Kreisschlüssel
0  Burg-St. Michaelisdonn                   10515163            1051
1           Marne-Nordsee                   10515166            1051
2                   Eider                   10515169            1051
3           Heider Umland                   10515172            1051
4      Mitteldithmarschen                   10515175            1051

Kreis:
                     Name  Kreisschlüssel  Landesschlüssel
0        Flensburg, Stadt            1001   

In [16]:
regionalebenen['Gemeinde']

,Gemeindeschlüssel,Name,Gemeindeverbandsschlüssel
0,10010000000,"Flensburg, Stadt",10010000
1,10020000000,"Kiel, Landeshauptstadt",10020000
2,10030000000,"Lübeck, Hansestadt",10030000
3,10040000000,"Neumünster, Stadt",10040000
4,10510011011,"Brunsbüttel, Stadt",10510011
...,...,...,...
10781,160775051011,Göpfersdorf,16077505
10782,160775051023,Langenleuba-Niederhain,16077505
10783,160775051036,Nobitz,16077505
10784,160775052003,Dobitschen,16077505


## restliche tabellen 

In [17]:
tabellen_liste = [
    'CSV_Klassenstufe',
    'CSV_Schulform',
    'CSV_Hoechster_Schulabschluss',
    'CSV_Hoechster_berufl_Abschluss',
    'CSV_Erwerbsstatus',
    'CSV_ET_Alter',
    'CSV_ET_Hoechst_berufl_Abschl',
    'CSV_ET_Stellung_im_Beruf',
    'CSV_ET_Beruf_Hauptgr_ISCO08',
    'CSV_ET_Wirtschaftszweig'
]

In [18]:
# Erstellen der Tabellen für alle definierten Tabellen und Konvertieren der Spalten
andere_tabellen = {}
for tabelle in tabellen_liste:
    df = created_dfs[tabelle].drop(columns=['Berichtszeitpunkt', 'Name', 'Regionalebene']).drop_duplicates().reset_index(drop=True)
    df['Schluessel'] = df['_RS'].astype(int)
    df = df.drop(columns=['_RS'])
    df = df.fillna(0)  # NA-Werte durch 0 ersetzen
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].apply(lambda x: int(x) if str(x).isdigit() else 0)
        else:
            df[col] = df[col].astype(int)
    # Verschieben der Schluessel-Spalte an die erste Position
    cols = ['Schluessel'] + [col for col in df if col != 'Schluessel']
    df = df[cols]
    andere_tabellen[tabelle] = df
    print(f"Tabelle {tabelle} wurde erfolgreich erstellt.")
    print(andere_tabellen[tabelle].head())


Tabelle CSV_Klassenstufe wurde erfolgreich erstellt.
    Schluessel  SCHUELER_KLSS_STP  SCHUELER_KLSS_STP__M  SCHUELER_KLSS_STP__W  \
0            0            8903780               4536040               4367740   
1            1             320060                162880                157180   
2         1001              10380                  5010                  5370   
3  10010000000              10380                  5010                  5370   
4         1002              24340                 12270                 12070   

   SCHUELER_KLSS_STP__1  SCHUELER_KLSS_STP__1_M  SCHUELER_KLSS_STP__1_W  \
0               3048570                 1563180                 1485400   
1                109240                   54930                   54310   
2                  3530                    1540                    1990   
3                  3530                    1540                    1990   
4                  8770                    4180                    4590   

   SCHUEL

In [19]:
# Anzeigen der erstellten Tabellen
print("\nErstellte Tabellen:")
for tabelle, df in andere_tabellen.items():
    print(f"\nTabelle {tabelle}:")
    print(df.head())


Erstellte Tabellen:

Tabelle CSV_Klassenstufe:
    Schluessel  SCHUELER_KLSS_STP  SCHUELER_KLSS_STP__M  SCHUELER_KLSS_STP__W  \
0            0            8903780               4536040               4367740   
1            1             320060                162880                157180   
2         1001              10380                  5010                  5370   
3  10010000000              10380                  5010                  5370   
4         1002              24340                 12270                 12070   

   SCHUELER_KLSS_STP__1  SCHUELER_KLSS_STP__1_M  SCHUELER_KLSS_STP__1_W  \
0               3048570                 1563180                 1485400   
1                109240                   54930                   54310   
2                  3530                    1540                    1990   
3                  3530                    1540                    1990   
4                  8770                    4180                    4590   

   SCHUELER_KL

In [20]:
andere_tabellen['CSV_ET_Wirtschaftszweig']

,Schluessel,ET_WIRTSZWG_STP,ET_WIRTSZWG_STP__M,ET_WIRTSZWG_STP__W,ET_WIRTSZWG_STP__1,ET_WIRTSZWG_STP__1_M,ET_WIRTSZWG_STP__1_W,ET_WIRTSZWG_STP__2,ET_WIRTSZWG_STP__2_M,ET_WIRTSZWG_STP__2_W,...,ET_WIRTSZWG_STP__33_W,ET_WIRTSZWG_STP__34,ET_WIRTSZWG_STP__34_M,ET_WIRTSZWG_STP__34_W,ET_WIRTSZWG_STP__35,ET_WIRTSZWG_STP__35_M,ET_WIRTSZWG_STP__35_W,ET_WIRTSZWG_STP__36,ET_WIRTSZWG_STP__36_M,ET_WIRTSZWG_STP__36_W
0,0,41047770,21622700,19425070,789080,534120,254960,10919270,8315160,2604110,...,756230,2874220,1552910,1321320,2995340,1399930,1595410,12115170,3566500,8548660
1,1,1479510,773840,705670,50420,34920,15510,332210,256660,75550,...,31820,82730,48350,34380,137240,69430,67810,402150,107770,294370
2,1001,48070,26040,22030,740,520,220,9760,7280,2480,...,680,2850,1750,1100,3990,2130,1860,14560,4810,9750
3,10010000000,48070,26040,22030,740,520,220,9760,7280,2480,...,680,2850,1750,1100,3990,2130,1860,14560,4810,9750
4,1002,128800,67060,61740,1510,960,550,20440,15950,4490,...,2180,10030,6070,3960,15100,7770,7340,41660,12850,28810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12434,160775051023,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12435,160775051036,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12436,160775052,6390,3590,2810,260,0,0,2200,1760,450,...,0,260,0,0,480,0,290,1570,470,1100
12437,160775052003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# export

In [31]:

db_path = r'C:\Users\hp\OneDrive\Desktop\DBU\SP3-1.Dateninfrastrukturen\Pruefung\Zensus\temp\zensus.db.session.sql'
if not os.path.exists(os.path.dirname(db_path)):
    raise FileNotFoundError(f"Der Pfad {os.path.dirname(db_path)} existiert nicht.")


In [32]:
# Verbindung zur SQLite-Datenbank herstellen
conn = sqlite3.connect(db_path)

In [33]:
# Regionalebenen-Tabellen in die SQLite-Datenbank übertragen
for ebene, df in regionalebenen.items():
    df.to_sql(f'dim_{ebene.lower()}', conn, if_exists='replace', index=False)

In [34]:
# Andere Tabellen in die SQLite-Datenbank übertragen
for tabelle, df in andere_tabellen.items():
    df.to_sql(f'dim_{tabelle.lower()}', conn, if_exists='replace', index=False)

In [35]:
conn.close()
print("DataFrames erfolgreich in die SQLite-Datenbank übertragen.")

DataFrames erfolgreich in die SQLite-Datenbank übertragen.
